In [ ]:
import pandas as pd
from numpy import *
from funcs import *
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')

##########################################################对训练集进行数据预处理####################################################
print 'loading train_set...'
# train_set = loadtxt('testdata.txt')
train_set1 = loadtxt('../data/sample_train.txt')
df = train_set_to_df(train_set1)

# 经纬度转换为坐标-----------------------------------------------------------------------------------------------------------
print 'transforming lon and lat to x and y...'
tmpx = []
tmpy = []
for point in df.itertuples():
    tmpx.append(point[3])
    tmpy.append(point[2])

df['x'] = tmpx
df['y'] = tmpy


## 1、求出每个点的速度、方向、速度较上一点的改变量、方向较上一点的改变量(v,theta,u,diata)-------------------------------------
print 'computing v and u...'
# 求出每个点的速度、方向(v,theta)-----------------------------------------

tmp = df
vtmp = [0]
thetatmp = [0]
vxtmp = [0]
vytmp = [0]
thetatmp = [0]

for i in range(1,len(tmp)):
    vx = tmp['x'][i] - tmp['x'][i-1]
    vxtmp.append(vx)
    vy = tmp['y'][i] - tmp['y'][i-1]    
    vytmp.append(vy)
    v = xy_to_len(vx,vy)
    vtmp.append(v)
    theta = tan_to_deg(vx,vy)
    thetatmp.append(theta)

df['vx'] = Series(vxtmp)
df['vy'] = Series(vytmp)
df['v'] = Series(vtmp)
df['theta'] = Series(thetatmp)

print 'fixing the error number...'
# 修复漏洞------------------------------------------------------------------------------------------------------------------------------
# 发现一个重大漏洞：每次计算没有把车辆分开计算，导致在index=800x处一系列数据异常，在index=800x+1处的加速度异常

# 为了保险起见，将df中的值传入dftest，如果成功，再把dftest传回df
dftest = df
# 每条轨迹的初始点
carnum = int(max(dftest['id'])+1)
tmp1 = dftest.loc[[car*800 for car in range(carnum)]]
# 每条轨迹的第二个点
# tmp2 = dftest.loc[[car*800+1 for car in range(carnum)]]

# 删除对应的错误行
dftest.drop([car*800 for car in range(carnum)],inplace=True)
# dftest.drop([car*800+1 for car in range(carnum)],inplace=True)

# 批量修改
tmp1['vx'] = 0.
tmp1['vy'] = 0.
tmp1['v'] = 0.
tmp1['theta'] = 0.

df = dftest.append(tmp1).sort_index()

# 保存修复后的df
df.to_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_df1.csv')
print 'df has been output'

# 提出用于参加训练隐马尔可夫模型的DataFrame：cdf-------------------------------------------------------------------------------------------------
# cdf = DataFrame([df['id'],df['x'],df['y'],df['v'],df['theta'],df['u'],df['diata'],df['stops_time']])
cdf = DataFrame([df['id'],df['x'],df['y'],df['v'],df['theta']])

cdf = cdf.transpose()
# fillna操作
cdf['theta'] = cdf['theta'].fillna(method='ffill')
# cdf['diata'] = cdf['diata'].fillna(0)
# 保存
cdf.to_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv')  
print 'cdf has been ouput'




##########################################################对测试集进行数据预处理####################################################
print 'loading test_set...'
# test_set = loadtxt('testdata2.txt')
test_set2 = loadtxt('/mnt/data2/TITS_data/geolife/geolife_test')
df = train_set_to_df(test_set2)

# 经纬度转换为坐标-----------------------------------------------------------------------------------------------------------------------------
print 'transforming lon and lat to x and y...'
tmpx = []
tmpy = []
for point in df.itertuples():
    tmpx.append(point[3])
    tmpy.append(point[2])

df['x'] = tmpx
df['y'] = tmpy

## 1、求出每个点的速度、方向、速度较上一点的改变量、方向较上一点的改变量(v,theta,u,diata)-------------------------------------
print 'computing v and u...'
# 求出每个点的速度、方向(v,theta)-----------------------------------------

tmp = df
vtmp = [0]
thetatmp = [0]
vxtmp = [0]
vytmp = [0]
thetatmp = [0]

for i in range(1,len(tmp)):
    vx = tmp['x'][i] - tmp['x'][i-1]
    vxtmp.append(vx)
    vy = tmp['y'][i] - tmp['y'][i-1]    
    vytmp.append(vy)
    v = xy_to_len(vx,vy)
    vtmp.append(v)
    theta = tan_to_deg(vx,vy)
    thetatmp.append(theta)

df['vx'] = Series(vxtmp)
df['vy'] = Series(vytmp)
df['v'] = Series(vtmp)
df['theta'] = Series(thetatmp)


print 'fixing the error number...'
# 修复漏洞------------------------------------------------------------------------------------------------------------------------------
# 发现一个重大漏洞：每次计算没有把车辆分开计算，导致在index=800x处一系列数据异常，在index=800x+1处的加速度异常

# 为了保险起见，将df中的值传入dftest，如果成功，再把dftest传回df
dftest = df
# 每条轨迹的初始点
carnum = int(max(dftest['id'])+1)
tmp1 = dftest.loc[[car*200 for car in range(carnum)]]
# 每条轨迹的第二个点
# tmp2 = dftest.loc[[car*200+1 for car in range(carnum)]]

# 删除对应的错误行
dftest.drop([car*200 for car in range(carnum)],inplace=True)
# dftest.drop([car*200+1 for car in range(carnum)],inplace=True)

# 批量修改
tmp1['vx'] = 0.
tmp1['vy'] = 0.
tmp1['v'] = 0.
tmp1['theta'] = 0.
df = dftest.append(tmp1).sort_index()





# 保存修复后的df
df.to_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_tdf1.csv')
print 'tdf has been output'

# 提出用于参加训练隐马尔可夫模型的DataFrame：cdf-------------------------------------------------------------------------------------------------
# cdf = DataFrame([df['id'],df['x'],df['y'],df['v'],df['theta'],df['u'],df['diata'],df['stops_time']])
cdf = DataFrame([df['id'],df['x'],df['y'],df['v'],df['theta']])

cdf = cdf.transpose()
# fillna操作
cdf['theta'] = cdf['theta'].fillna(method='ffill')
# cdf['diata'] = cdf['diata'].fillna(0)
# 保存
cdf.to_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_ctdf1.csv')  
print 'ctdf has been ouput'


print 'data_pretreatment has been done'




model_geolife_tongyi_300_iter_v_1m15d = hmm.GaussianHMM(n_components=16,n_iter=300)

cdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)
traincol = cdf[['v']]
traincol = np.array(traincol)
lengths = [800]*(len(traincol)/800)
model_geolife_tongyi_300_iter_v_1m15d.fit(traincol,lengths=lengths)
joblib.dump(model_geolife_tongyi_300_iter_v_1m15d,'../models/model_geolife_tongyi_300_iter_v_1m15d.pkl')

loading train_set...
transforming lon and lat to x and y...
computing v and u...
fixing the error number...
df has been output
cdf has been ouput
loading test_set...
transforming lon and lat to x and y...
computing v and u...
fixing the error number...
tdf has been output
ctdf has been ouput
data_pretreatment has been done


In [ ]:
from tqdm import tqdm_notebook as tqdm

df = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_df1.csv',index_col=0)   # index_col=0 可以去掉Unnamed0
cdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)
tdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_tdf1.csv',index_col=0)
ctdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)

state_to_v = state_mean_v(model_geolife_tongyi_300_iter_v_1m15d,cdf)

# 假·隐马尔可夫模型-----------------------------------------------------------------------------------------
# 预测一次，后面等距

print '假·隐马尔可夫模型'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '假·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '假·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '假·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '假·step=30',mean(errors)

# 真·隐马尔可夫模型-------------------------------------------------------------------------------------------
# 每步的预测严格依照上一步预测

print '真·隐马尔可夫模型'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '真·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '真·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '真·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '真·step=30',mean(errors)

假·隐马尔可夫模型
1 step distance error


HBox(children=(IntProgress(value=0, max=150), HTML(value=u'')))


假·step=1 0.000113453324507
10 step distance error


HBox(children=(IntProgress(value=0, max=140), HTML(value=u'')))


假·step=10 0.000939887413904
20 step distance error


HBox(children=(IntProgress(value=0, max=130), HTML(value=u'')))


假·step=20 0.00199607162065
30 step distance error


HBox(children=(IntProgress(value=0, max=120), HTML(value=u'')))


假·step=30 0.00314326577344
真·隐马尔可夫模型
1 step distance error


HBox(children=(IntProgress(value=0, max=150), HTML(value=u'')))

In [7]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from numpy import *
from funcs import *
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')
%run funcs.py

model_geolife_tongyi_300_iter_v_1m15d = joblib.load('../models/model_geolife_tongyi_300_iter_v_1m15d.pkl')

from tqdm import tqdm_notebook as tqdm

df = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_df1.csv',index_col=0)   # index_col=0 可以去掉Unnamed0
cdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)
tdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_tdf1.csv',index_col=0)
ctdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)

state_to_v = state_mean_v(model_geolife_tongyi_300_iter_v_1m15d,cdf)

# 真·隐马尔可夫模型-------------------------------------------------------------------------------------------
# 每步的预测严格依照上一步预测

print '真·隐马尔可夫模型'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '真·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '真·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '真·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '真·step=30',mean(errors)

真·隐马尔可夫模型
1 step distance error


HBox(children=(IntProgress(value=0, max=150), HTML(value=u'')))


真·step=1 0.00011345332450746011
10 step distance error


HBox(children=(IntProgress(value=0, max=140), HTML(value=u'')))


真·step=10 0.0010022125386457506
20 step distance error


HBox(children=(IntProgress(value=0, max=130), HTML(value=u'')))


真·step=20 0.0021369927710491263
30 step distance error


HBox(children=(IntProgress(value=0, max=120), HTML(value=u'')))


真·step=30 0.003366791081903485


In [1]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from numpy import *
from funcs import *
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')
%run funcs.py

model_geolife_tongyi_300_iter_v_1m15d = joblib.load('../models/model_geolife_tongyi_300_iter_v_1m15d.pkl')

cdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_cdf1.csv',index_col=0)
ctdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_ctdf1.csv',index_col=0)


state_to_v = state_mean_v(model_geolife_tongyi_300_iter_v_1m15d,cdf)

print '假·隐马尔可夫模型'

print '40 step distance error'
errors = []
for timestamp in tqdm(range(29,139)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=40))
print '假·step=40',mean(errors)


print '50 step distance error'
errors = []
for timestamp in tqdm(range(29,129)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=50))
print '假·step=50',mean(errors)

# 真·隐马尔可夫模型-------------------------------------------------------------------------------------------
# 每步的预测严格依照上一步预测

print '真·隐马尔可夫模型'

print '40 step distance error'
errors = []
for timestamp in tqdm(range(29,139)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=40))
print '真·step=40',mean(errors)


print '50 step distance error'
errors = []
for timestamp in tqdm(range(29,129)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=50))
print '真·step=50',mean(errors)

假·隐马尔可夫模型
40 step distance error


HBox(children=(IntProgress(value=0, max=110), HTML(value=u'')))


假·step=40 0.00999355965781159
50 step distance error


HBox(children=(IntProgress(value=0), HTML(value=u'')))


假·step=50 0.013399646735105918
真·隐马尔可夫模型
40 step distance error


HBox(children=(IntProgress(value=0, max=110), HTML(value=u'')))


真·step=40 0.010318846614337948
50 step distance error


HBox(children=(IntProgress(value=0), HTML(value=u'')))


真·step=50 0.013803129348388697


In [1]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from numpy import *
from funcs import *
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')
%run funcs.py

model_geolife_tongyi_300_iter_v_1m15d = joblib.load('../models/model_geolife_tongyi_300_iter_v_1m15d.pkl')

# df = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_df1.csv',index_col=0)   # index_col=0 可以去掉Unnamed0
# cdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)
# tdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_tdf1.csv',index_col=0)
# ctdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)

cdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_cdf1.csv',index_col=0)
ctdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_ctdf1.csv',index_col=0)


state_to_v = state_mean_v(model_geolife_tongyi_300_iter_v_1m15d,cdf)

# 假·隐马尔可夫模型-----------------------------------------------------------------------------------------
# 预测一次，后面等距

print '假·隐马尔可夫模型'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '假·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '假·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '假·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '假·step=30',mean(errors)

print '40 step distance error'
errors = []
for timestamp in tqdm(range(29,139)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=40))
print '假·step=40',mean(errors)


print '50 step distance error'
errors = []
for timestamp in tqdm(range(29,129)):
    errors.append(over_error(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=50))
print '假·step=50',mean(errors)

# 真·隐马尔可夫模型-------------------------------------------------------------------------------------------
# 每步的预测严格依照上一步预测

print '真·隐马尔可夫模型'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '真·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '真·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '真·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '真·step=30',mean(errors)


print '40 step distance error'
errors = []
for timestamp in tqdm(range(29,139)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=40))
print '真·step=40',mean(errors)


print '50 step distance error'
errors = []
for timestamp in tqdm(range(29,129)):
    errors.append(over_error1(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=50))
print '真·step=50',mean(errors)

假·隐马尔可夫模型
1 step distance error


HBox(children=(IntProgress(value=0, max=150), HTML(value=u'')))


假·step=1 0.00012575059120513202
10 step distance error


HBox(children=(IntProgress(value=0, max=140), HTML(value=u'')))


假·step=10 0.0019974101236076265
20 step distance error


...


真·step=30 0.007254877861939086
40 step distance error


HBox(children=(IntProgress(value=0, max=110), HTML(value=u'')))


真·step=40 0.010318846614337948
50 step distance error


HBox(children=(IntProgress(value=0), HTML(value=u'')))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from numpy import *
from funcs import *
from pandas import Series,DataFrame
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')
%run funcs.py

model_geolife_tongyi_300_iter_v_1m15d = joblib.load('../models/model_geolife_tongyi_300_iter_v_1m15d.pkl')

# df = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_df1.csv',index_col=0)   # index_col=0 可以去掉Unnamed0
# cdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)
# tdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_tdf1.csv',index_col=0)
# ctdf = pd.read_csv('/mnt/data1/liheyuan/MD/HMM/hmm4/geolife_tongyi_cdf1.csv',index_col=0)

cdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_cdf1.csv',index_col=0)
ctdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/geolife_tongyi_ctdf1.csv',index_col=0)


state_to_v = state_mean_v(model_geolife_tongyi_300_iter_v_1m15d,cdf)

# 假·隐马尔可夫模型-----------------------------------------------------------------------------------------
# 预测一次，后面等距

print '假·隐马尔可夫模型，测试集长度200'

print '1 step distance error'
errors = []
for timestamp in tqdm(range(29,179)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=1))
print '假·step=1',mean(errors)


print '10 step distance error'
errors = []
for timestamp in tqdm(range(29,169)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=10))
print '假·step=10',mean(errors)


print '20 step distance error'
errors = []
for timestamp in tqdm(range(29,159)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=20))
print '假·step=20',mean(errors)


print '30 step distance error'
errors = []
for timestamp in tqdm(range(29,149)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=30))
print '假·step=30',mean(errors)

print '40 step distance error'
errors = []
for timestamp in tqdm(range(29,139)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=40))
print '假·step=40',mean(errors)


print '50 step distance error'
errors = []
for timestamp in tqdm(range(29,129)):
    errors.append(over_error_200(model_geolife_tongyi_300_iter_v_1m15d,cdf,ctdf,last_point_num=timestamp,steps=50))
print '假·step=50',mean(errors)

假·隐马尔可夫模型，测试集长度200
1 step distance error


HBox(children=(IntProgress(value=0, max=150), HTML(value=u'')))


假·step=1 0.00012407890797876716
10 step distance error


HBox(children=(IntProgress(value=0, max=140), HTML(value=u'')))


假·step=10 0.0006036500218549198
20 step distance error


...


假·step=30 0.0018687654586913816
40 step distance error


HBox(children=(IntProgress(value=0, max=110), HTML(value=u'')))


假·step=40 0.0025207577596830203
50 step distance error


HBox(children=(IntProgress(value=0), HTML(value=u'')))


假·step=50 0.0031791312281895003
